<a href="https://colab.research.google.com/github/Crazydodo123/code-ml24/blob/main/nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

In [2]:
import pandas as pd

air_canada = pd.read_csv('./new_data.csv')
air_canada = air_canada.dropna(subset=['choice'])

y = air_canada['choice']
X = air_canada.drop(['id', 'Unnamed: 0','flight_departure_datetime', 'purchase_datetime', 'ticket_id', 'choice'], axis=1)

In [3]:
from sklearn.model_selection import train_test_split

# Split data into train (80%), validation (10%), and test (10%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [6]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_pipeline = make_pipeline(
  StandardScaler()
)

cat_pipeline = make_pipeline(
  OneHotEncoder()
)

preprocessing = ColumnTransformer([
    ('cat', cat_pipeline, ['od']),
    ('num', num_pipeline, ['date_difference', 'trip_type', 'branded_fare', 'number_of_pax', 'ADVS_price', 'PREF_price',
       'ADVS_capacity', 'PREF_capacity', 'ADVS_inventory', 'PREF_inventory'])
  ], remainder='drop')

In [7]:
X_train_prep = preprocessing.fit_transform(X_train)
X_val_prep = preprocessing.fit_transform(X_val)
X_test_prep = preprocessing.transform(X_test)

In [13]:
# One-hot encode the y (choice) column
from sklearn.preprocessing import OneHotEncoder

# One-hot encode the y (choice) column
encoder = OneHotEncoder(sparse_output=False)

y_train_prep = encoder.fit_transform(pd.DataFrame(y_train))
y_val_prep = encoder.fit_transform(pd.DataFrame(y_val))
y_test_prep = encoder.fit_transform(pd.DataFrame(y_test))

In [16]:
# Convert data to TensorFlow tensors
X_train_tensor = tf.convert_to_tensor(X_train_prep, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train_prep, dtype=tf.float32)
X_val_tensor = tf.convert_to_tensor(X_val_prep, dtype=tf.float32)
y_val_tensor = tf.convert_to_tensor(y_val_prep, dtype=tf.float32)
X_test_tensor = tf.convert_to_tensor(X_test_prep, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test_prep, dtype=tf.float32)

In [27]:
X_train_tensor.shape, X_val_tensor.shape, X_test_tensor.shape

(TensorShape([153998, 20]), TensorShape([19250, 20]), TensorShape([19250, 20]))

In [30]:
# Create a basic TensorFlow Sequential model that takes input with 20 features
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(20,)),  # Input layer expecting 20 features per row
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # Output layer with 4 choices (assuming 4 classes in 'choice')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(X_train_tensor, y_train_tensor, epochs=20, validation_data=(X_val_tensor, y_val_tensor))

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test_tensor, y_test_tensor)
print(f"Test accuracy: {test_accuracy:.2f}")